In [1]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.ml.classification import LogisticRegression
from pyspark.context import SparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.session import SparkSession
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler


spark = SparkSession.builder.master("local").appName("History").getOrCreate()

In [2]:
df = spark.read.format("text").option("header", "true").option("inferSchema", "true").load("Qualitative_Bankruptcy.txt")
df.show()

+--------------+
|         value|
+--------------+
|P,P,A,A,A,P,NB|
|N,N,A,A,A,N,NB|
|A,A,A,A,A,A,NB|
|P,P,P,P,P,P,NB|
|N,N,P,P,P,N,NB|
|A,A,P,P,P,A,NB|
|P,P,A,P,P,P,NB|
|P,P,P,A,A,P,NB|
|P,P,A,P,A,P,NB|
|P,P,A,A,P,P,NB|
|P,P,P,P,A,P,NB|
|P,P,P,A,P,P,NB|
|N,N,A,P,P,N,NB|
|N,N,P,A,A,N,NB|
|N,N,A,P,A,N,NB|
|N,N,A,P,A,N,NB|
|N,N,A,A,P,N,NB|
|N,N,P,P,A,N,NB|
|N,N,P,A,P,N,NB|
|A,A,A,P,P,A,NB|
+--------------+
only showing top 20 rows



In [3]:
df.count()

250

In [4]:
get_Double_Value = { 'P' : 3.0, 'A' : 2.0, 'N' : 1.0, 'NB': 1.0, 'B': 0.0 }    

In [5]:
df.collect()[0][0]

'P,P,A,A,A,P,NB'

In [11]:
def line_parser(line):
    tokens_df = line.split(',')
    label_df = get_Double_Value[tokens_df[-1]]
    features_df = list(map(lambda t: get_Double_Value[t], tokens_df[:-1]))
    return LabeledPoint(label_df, features_df)

parsed_d= df.rdd.map(lambda x: line_parser(x[0]))
parsed_d

PythonRDD[88] at RDD at PythonRDD.scala:53

In [13]:
print(parsed_d.take(1)[0])
train_d, test_d = parsed_d.randomSplit([0.6, 0.4], seed = 0)

model_fit = LogisticRegressionWithLBFGS.train(train_d, numClasses = 2)

(1.0,[3.0,3.0,2.0,2.0,2.0,3.0])


In [15]:
label_pred = test_d.map(lambda point: (point.label,  model_fit.predict(point.features)))

In [18]:
train_error = len(list(filter(lambda r: (r[0] != r[1]), label_pred.collect())))/test_d.count()
print(f"Training Error Rate = {train_error}")

Training Error Rate = 0.2037037037037037
